In [1]:
import os

In [2]:
pwd

'c:\\Users\\dvidyara\\OneDrive - Cisco\\Desktop\\MLEND-END\\MLFlow-END-END-Project\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\dvidyara\\OneDrive - Cisco\\Desktop\\MLEND-END\\MLFlow-END-END-Project'

In [69]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dhanushgowda91/MLFlow-END-END-Project.mlflow"
os.environ["MFLOW_TRACKING_USERNAME"]="dhanushgowda91"
os.environ["MLFLOW_TRACKING_PASSWORD"]="da6e989a9a3a41e914c03a04ebd5e04880087fa5"

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaulationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [71]:
pip install dagshub

In [72]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml , create_directories

class ConfigurationManager:

    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath= SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaulationConfig:
        config= self.config.model_evaluation
        params= self.params.ElasticNet
        schema= self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config= ModelEvaulationConfig(root_dir=config.root_dir,test_data_path=config.test_data_path,model_path=config.model_path,all_params=params,
            metric_file_name= config.metric_file_name,target_column=schema.name,mlflow_uri="https://dagshub.com/dhanushgowda91/MLFlow-END-END-Project.mlflow")
        return model_evaluation_config



In [73]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [74]:
from mlProject.utils.common import save_json

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dhanushgowda91/MLFlow-END-END-Project.mlflow"
os.environ["MFLOW_TRACKING_USERNAME"]="dhanushgowda91"
os.environ["MLFLOW_TRACKING_PASSWORD"]="da6e989a9a3a41e914c03a04ebd5e04880087fa5"



class ModelEvaulation:
    def __init__(self, config: ModelEvaulationConfig):
        self.config=config

    def eval_metrics(self, actual, pred):
        rmse= np.sqrt(mean_squared_error(actual, pred))
        mae= mean_absolute_error(actual, pred)
        r2= r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data= pd.read_csv(self.config.test_data_path)
        model= joblib.load(self.config.model_path)

        test_x= test_data.drop([self.config.target_column], axis=1)
        test_y= test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=  model.predict(test_x)
            (rmse, mae,r2)= self.eval_metrics(test_y, predicted_qualities)
            scores= {"rmse": rmse, "mae": mae , "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data= scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

        
        if tracking_url_type_score != "file":
            mlflow.sklearn.log_model(model, "model", registered_model_name= "ElasticnetModel")
        else:
             mlflow.sklearn.log_model(model, "model")



In [75]:
try:
    config= ConfigurationManager()
    data_evaluation_config= config.get_model_evaluation_config()
    data_evaluation_config= ModelEvaulation(config=data_evaluation_config)
    data_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e 

[2024-07-02 17:11:10,553: INFO: common:  yaml file: config\config.yaml loaded successfully]
[2024-07-02 17:11:10,557: INFO: common:  yaml file: params.yaml loaded successfully]
[2024-07-02 17:11:10,562: INFO: common:  yaml file: schema.yaml loaded successfully]
[2024-07-02 17:11:10,564: INFO: common: Created directory at: artifacts]
[2024-07-02 17:11:10,566: INFO: common: Created directory at: artifacts/model_evaluation]


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 403 != 200. Response body: ''